In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

base_path = '/content/drive/MyDrive/mvtec_project/metal_nut'
train_dir = os.path.join(base_path, 'train', 'images')
test_dir = os.path.join(base_path, 'test', 'images')

print("Train directory:", train_dir)
print("Exists:", os.path.exists(train_dir))
print("Test directory:", test_dir)
print("Exists:", os.path.exists(test_dir))

In [ ]:
!pip install -q torch torchvision matplotlib

In [ ]:
import os

test_normal_dir = '/content/drive/MyDrive/mvtec_project/metal_nut/test/images/normal'
files = os.listdir(test_normal_dir) if os.path.exists(test_normal_dir) else []
print(f"Files in test 'normal' folder ({test_normal_dir}): {files[:10]}")  # Show first 10 files or empty

In [ ]:
import os
import shutil

def wrap_test_images(image_dir):
    normal_dir = os.path.join(image_dir, 'normal')
    if not os.path.exists(normal_dir):
        os.makedirs(normal_dir)
    for f in os.listdir(image_dir):
        fpath = os.path.join(image_dir, f)
        if os.path.isfile(fpath) and f.lower().endswith(('.png', '.jpg', '.jpeg')):
            shutil.move(fpath, os.path.join(normal_dir, f))

wrap_test_images('/content/drive/MyDrive/mvtec_project/metal_nut/test/images')
print("✅ Moved test images into 'normal' folder.")


✅ Moved test images into 'normal' folder.


In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# Define transform
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Set paths
train_dir = '/content/drive/MyDrive/mvtec_project/metal_nut/train/images'
test_dir = '/content/drive/MyDrive/mvtec_project/metal_nut/test/images'

# Load datasets
train_dataset = ImageFolder(root=train_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print(f"Train classes: {train_dataset.classes}")
print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")


Train classes: ['normal']
Train samples: 50
Test samples: 25


#Step-by-Step Autoencoder-Based Anomaly Detection with Reconstruction Visualization

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Autoencoder architecture
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1),  # 64x64
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1), # 32x32
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1), # 16x16
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1), # 32x32
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1), # 64x64
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 3, stride=2, padding=1, output_padding=1),  # 128x128
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize model, loss, optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ConvAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
n_epochs = 20
for epoch in range(n_epochs):
    model.train()
    train_loss = 0
    for images, _ in train_loader:
        images = images.to(device)
        outputs = model(images)
        loss = criterion(outputs, images)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {train_loss / len(train_loader):.4f}")

# Evaluate reconstruction error on test images
model.eval()
import numpy as np

def plot_reconstructions(model, loader, num_images=5):
    data_iter = iter(loader)
    images, _ = next(data_iter)
    images = images[:num_images].to(device)
    with torch.no_grad():
        outputs = model(images)

    fig, axes = plt.subplots(2, num_images, figsize=(15, 4))
    for i in range(num_images):
        axes[0, i].imshow(images[i].cpu().permute(1, 2, 0))
        axes[0, i].set_title("Original")
        axes[0, i].axis('off')

        axes[1, i].imshow(outputs[i].cpu().permute(1, 2, 0))
        axes[1, i].set_title("Reconstructed")
        axes[1, i].axis('off')
    plt.tight_layout()
    plt.show()

plot_reconstructions(model, test_loader)

#Visualizing Anomalies Using Error Heatmaps: Comparing Normal and Defective Images

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import os
from PIL import Image

# Paths
test_root = '/content/drive/MyDrive/mvtec_project/metal_nut/test'

# Transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Load test dataset (normal and defective)
test_dataset = ImageFolder(root=test_root, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Class mapping
print("Class to index:", test_dataset.class_to_idx)

# Autoencoder model (assume it's called 'model')
model.eval()

# Function to compute heatmap
def show_anomaly(original, reconstructed):
    error = torch.abs(original - reconstructed).squeeze().detach().cpu().numpy()
    heatmap = np.mean(error, axis=0)  # mean over channels
    return heatmap

# Visualize few results
for i, (img, label) in enumerate(test_loader):
    if i >= 5: break  # just show 5 images

    img = img.to(device)
    with torch.no_grad():
        reconstructed = model(img)

    heatmap = show_anomaly(img, reconstructed)

    # Convert to numpy for plotting
    original_np = img.squeeze().permute(1, 2, 0).cpu().numpy()
    reconstructed_np = reconstructed.squeeze().permute(1, 2, 0).cpu().numpy()

    # Plot
    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    axs[0].imshow(original_np)
    axs[0].set_title("Original")
    axs[1].imshow(reconstructed_np)
    axs[1].set_title("Reconstructed")
    axs[2].imshow(heatmap, cmap='hot')
    axs[2].set_title("Anomaly Heatmap")
    for ax in axs: ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from torchvision.utils import save_image

# Make sure your model and device are defined
model.eval()

output_dir = '/content/drive/MyDrive/mvtec_project/results'
os.makedirs(output_dir, exist_ok=True)

def calculate_threshold(errors, k=3):
    mean = np.mean(errors)
    std = np.std(errors)
    return mean + k * std

anomaly_scores = []
labels = []

for i, (img, label) in enumerate(test_loader):
    img = img.to(device)
    with torch.no_grad():
        reconstructed = model(img)

    error_map = (img - reconstructed) ** 2
    error_score = error_map.mean().item()

    anomaly_scores.append(error_score)
    labels.append(label.item())

    # Save images
    save_image(img.cpu(), os.path.join(output_dir, f'{i}_original.png'))
    save_image(reconstructed.cpu(), os.path.join(output_dir, f'{i}_reconstructed.png'))

    heatmap = error_map.squeeze().mean(dim=0).cpu().numpy()
    plt.imshow(heatmap, cmap='hot')
    plt.axis('off')
    plt.title(f'Error Heatmap (Score: {error_score:.4f})')
    plt.savefig(os.path.join(output_dir, f'{i}_heatmap.png'))
    plt.close()

print(f"Saved results to {output_dir}")

# Calculate threshold from normal images
normal_class_idx = test_dataset.class_to_idx['normal']
normal_scores = np.array(anomaly_scores)[np.array(labels) == normal_class_idx]
threshold = calculate_threshold(normal_scores, k=3)
print(f"Anomaly score threshold: {threshold:.4f}")

# Predict anomalies
predicted = np.array(anomaly_scores) > threshold
true_labels = np.array(labels) != normal_class_idx  # normal=0, defective=1 (currently you only have normal, so all 0)

# Accuracy calculation (won't be meaningful until you add defective images)
accuracy = (predicted == true_labels).mean()
print(f"Anomaly detection accuracy: {accuracy:.4f}")


Saved results to /content/drive/MyDrive/mvtec_project/results
Anomaly score threshold: 0.0093
Anomaly detection accuracy: 1.0000


In [ ]:
model_path = '/content/drive/MyDrive/mvtec_project/autoencoder_model.pth'
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/mvtec_project/autoencoder_model.pth
